<a href="https://colab.research.google.com/github/SaketMunda/introduction-to-nlp/blob/master/nlp_with_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Natural Language Processing with TensorFlow

NLP has the goal of deriving information out of natural language (could be sequences text or speech).

Another common term for NLP problems is sequence to sequence problems (seq2seq).

In [1]:
# Since we're going to experiment deep-learning models so we need to enable GPUs
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-0c0b432e-a08e-be39-c90b-2ff3f528f4e6)


## Get Helper functions

In [2]:
# Get helper_functions.py script from Github
!wget https://raw.githubusercontent.com/SaketMunda/ml-helpers/master/helper_functions.py

from helper_functions import unzip_data, create_tensorboard_callback

--2023-01-30 05:04:16--  https://raw.githubusercontent.com/SaketMunda/ml-helpers/master/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2904 (2.8K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]   2.84K  --.-KB/s    in 0s      

2023-01-30 05:04:16 (45.4 MB/s) - ‘helper_functions.py’ saved [2904/2904]



## Get a Text Dataset
The dataset that we're going to be using is Kaggle's introduction to NLP dataset (text samples of Tweets labelled as disaster or not disaster)

See the original source here: https://www.kaggle.com/c/nlp-getting-started

In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

# unzip the data
unzip_data('nlp_getting_started.zip')

--2023-01-30 05:04:21--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.68.128, 74.125.24.128, 142.250.4.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.68.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.02s   

2023-01-30 05:04:22 (28.9 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualizing Text Dataset

To visualize our text samples, we first have to read them in, so we can do it through pandas.

In [4]:
import pandas as pd
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# check the shapes
train_df.shape, test_df.shape

((7613, 5), (3263, 4))

In [ ]:
# view some samples
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


So here, `text` is the tweet and `target` variable is to identify whether the tweet is a disaster or not, so if `1` then it's a disaster else not a disaster.

Let's visualize some random `training` samples, but before that this is a good practice to shuffle the training samples first,

In [5]:
train_df_shuffled = train_df.sample(frac=1, random_state=17)
# frac=1 means 100% of samples will be shuffled
train_df_shuffled

,id,keyword,location,text,target
7027,10072,typhoon,NaN,Typhoon Soudelor: When will it hit Taiwan ÛÒ ...,1
318,463,armageddon,NaN,RT @RTRRTcoach: #Love #TrueLove #romance lith ...,0
1681,2425,collide,www.youtube.com?Malkavius2,I liked a @YouTube video from @gassymexican ht...,0
5131,7318,nuclear%20reactor,"New York, New York",Japan's Restart of Nuclear Reactor Fleet Fast ...,1
2967,4262,drowning,"Hendersonville, NC",#ICYMI #Annoucement from Al Jackson... http://...,0
...,...,...,...,...,...
406,584,arson,"Jerusalem, Israel",Mourning notices for stabbing arson victims st...,1
5510,7863,quarantined,"Livonia, MI",Reddit's new content policy goes into effect m...,0
2191,3139,debris,NaN,Plane debris discovered on Reunion Island belo...,1
7409,10600,wounded,santo domingo,Police Officer Wounded Suspect Dead After Exch...,1


In [ ]:
# how does the test set looks like ?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [ ]:
# How many examples of each class ?
train_df['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [ ]:
# Let's visualize some random samples

import random
random_index = random.randint(0, len(train_df_shuffled)-5)
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(disaster)" if target > 0 else "(not a disaster)")
  print(f"Text:\n{text}\n")
  print("----------\n")

Target: 1 (disaster)
Text:
@mylittlepwnies3 @Early__May @AnathemaZhiv @TonySandos much of which has to do with lebanon 80s attack/ iran hostage crisis/ Libya Pan am

----------

Target: 0 (not a disaster)
Text:
Oil prices falling but drivers could reap benefits http://t.co/QlTwhoJqYA

----------

Target: 1 (disaster)
Text:
Gunmen open fire on bus near El Salvador's capital killing 4 a week after gang attacks killed 8 bus drivers: http://t.co/Pz56zJSsfT bitÛ_

----------

Target: 0 (not a disaster)
Text:
hermancranston: #atk #LetsFootball RT SkanndTyagi: #letsFootball #atk WIRED : All these fires are burning through Û_ http://t.co/DmTab6g7j7

----------

Target: 1 (disaster)
Text:
Police Officer Wounded Suspect Dead After Exchanging Shots http://t.co/brE2lGmn7C #ABC #News #AN247

----------



## Split dataset into Train and Validation sets

Since the test set doesn't contain the target variable so we might need some unseen data for model to be validated after training, so how about splitting our training set for validating purpose with some amount.


In [6]:
from sklearn.model_selection import train_test_split

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=17)

## Converting Text into Numbers

Our labels are in numerical form (0 and 1) but our tweets are in string form.

But machine learning algorithm learns only through numbers so we have to convert those tweets/texts into numbers.

In NLP, there are two main concepts for turning text into numbers,
- **Tokenization** : A straight mapping from **word**(known as *word-level tokenization*) or character(which is *character-level tokenization*) or sub-word(*sub-word tokenization*) to a numerical value. Just like One hot encoding, suppose we have a sentence as "My name is Alpha", then if we are mapping according to word, "My" would `0`, "name" as `1`, "is" as `2` and "Alpha" as `3`.
- **Embeddings** : An embedding is a representation of natural language which can be learned. Representation comes in the form of **feature-vector**. For example the word "Alpha" could be represented by 5-D vector `[0.564, 0.897, 0.456, -0.987, 0.15]`. The size of the feature vector is tuneable. There are two ways to use embeddings:

    - **Create your own embedding** - Once your text has been turned into numbers (required for an embedding), you can put them through an embedding layer (such as `tf.keras.layers.Embedding`) and an embedding representation will be learned during model training.
    - **Reuse a pre-learned embedding** - Many pre-trained embeddings exist online. These pre-trained embeddings have often been learned on large corpuses of text (such as all of Wikipedia) and thus have a good underlying representation of natural language. You can use a pre-trained embedding to initialize your model and fine-tune it to your own specific task.


Simply, 

**Tokenization** : Straight mapping from word to number.

**Embedding** : Richer representation of relationships between tokens.

It depends on your problem. You could try character-level tokenization/embeddings and word-level tokenization/embeddings and see which perform best. You might even want to try stacking them (e.g. combining the outputs of your embedding layers using [tf.keras.layers.concatenate](https://www.tensorflow.org/api_docs/python/tf/keras/layers/concatenate)).

If you're looking for pre-trained word embeddings, [Word2vec embeddings](https://jalammar.github.io/illustrated-word2vec/), [GloVe embeddings](https://nlp.stanford.edu/projects/glove/) and many of the options available on TensorFlow Hub are great places to start.

Much like searching for a pre-trained computer vision model, we can search for pre-trained word embedding to use for your problem. Try searching for something like "use pre-trained word embeddings in TensorFlow".

### Text Vectorization

Mapping words to numbers.

To tokenize our words, we'll use the preprocessing layer,
`tf.keras.layers.preprocessing.TextVectorization`

In [7]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Using the default TextVectorization variables
text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary (all of the different words in your text)
                                 standardize="lower_and_strip_punctuation", # how to process the text
                                 split="whitespace", # how to split the text
                                 ngrams=None, # create groups of n-words
                                 output_mode='int', # how to map tokens to numbers
                                 output_sequence_length=None) # How long should the output sequence of tokens be?

About the above params,

- `max_tokens` : The maximum number of words in your vocabulary (e.g 20000 or the number of unique words in your text), includes a value for OOV(out of vocabulary) tokens
- `standardize` : Methods for standardizing text
- `split`: split the text
- `ngrams`: how many words to contain per token split, for example if 2, it splits tokens into continous sequences of 2
- `output_mode`: How to output tokens can be `int`(integer mapping), `binary`(OHE), `count` or `tf-idf`
- `output_sequence_length`: Length of tokenized sequence to output, For example if set to 150, all tokenized sequences will be 150 tokens long.

In the above cell, we have initialized the object with the default settings but let's customize it a little bit for our own use case.

In particular, let's set values for `max_tokens` and `output_sequence_length`.

For `max_tokens`(the number of words in the vocabulary), multiples of 10,000(`10,000`, `20,000`, `30,000`) or the exact number of unqiue words in your text(e.g `32,179`) are common values.

For our use case, `10,000`

And for the `output_sequence_length` we'll use the average number of tokens per Tweet in the training set. But first, we'll need to find it.

In [ ]:
# Find average number of tokens (words) in training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [8]:
# Setup text vectorization with custom variables
max_vocab_length = 10000
max_length = 15

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode='int',
                                    output_sequence_length=max_length)

To map our `TextVectorization` instance `text_vectorizer` to our data, we can call the `adapt()` method on it whilst passing it our training set.

In [9]:
# fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

Training data mapped! Let's try our `text_vectorizer` on a custom sentence.

In [10]:
# create a sample sentence
sample_sentence = "There's a flood in my village!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[281,   3, 214,   4,  13, 881,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

Try our `text_vectorizer` on a few random sentences ?

In [13]:
import random
random_sentence = random.choice(train_sentences)
print(f'Original Text: \n{random_sentence}\
        \n\n Vectorized Text:')
text_vectorizer([random_sentence])

Original Text: 
11-Year-Old Boy Charged With Manslaughter of Toddler: Report: An 11-year-old boy has been charged with manslaughter over the fatal sh...        

 Vectorized Text:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[823, 347, 389,  14, 727,   6, 828, 407,  41, 823, 347,  40,  64,
        389,  14]])>

We can also check the unique tokens in our vocabulary using the `get_vocabulary()` method

In [14]:
# get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5] # most common tokens (notice the [UNK] token for "unknown" words)
bottom_5_words = words_in_vocab[-5:] # least common tokens

print(f"Number of words in Vocab:{len(words_in_vocab)}")
print(f"Top 5 most common words:{top_5_words}")
print(f"Bottom 5 least common words:{bottom_5_words}")

Number of words in Vocab:10000
Top 5 most common words:['', '[UNK]', 'the', 'a', 'in']
Bottom 5 least common words:['ovo', 'overåÊhostages', 'overzero', 'overwatch', 'overturns']


### Embedding

**Create an Embedding using Embedding Layer**

We've got a way to map our text to numbers. How about we go a step further and turn those numbers into an embedding?

The powerful thing about an embedding is it can be learned during training. This means rather that just be static, a word's numeric representation can be improved as a model goes through data samples.

We can see what an embedding of a word looks like by using the `tf.keras.layers.Embedding` layer.



In [15]:
tf.random.set_seed(17)
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             embeddings_initializer="uniform",
                             input_length=max_length,
                             name="embedding_1")
embedding

`embedding` is a TensorFlow Layer, so that we can use it as part of a model, meaning its parameters(word representations) can be updated and improved as the model learns.



In [16]:
# Get a random sentence from training set
random_sentence = random.choice(train_sentences)
print(f"Original text: \n{random_sentence}")
print("\n\nEmbedded version:")

# embed the random sentence
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text: 
Now that's what you call a batting collapse #theashes


Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.01314962,  0.00719387,  0.02004207, ..., -0.04191788,
          0.0154699 , -0.01650833],
        [-0.00079762,  0.04007155,  0.01404953, ...,  0.02520284,
          0.0242764 ,  0.03214594],
        [ 0.01604157, -0.00236765,  0.01929165, ...,  0.03563354,
          0.04398265, -0.03589463],
        ...,
        [-0.04305307, -0.04987746, -0.0366137 , ...,  0.01739735,
          0.00826229,  0.04193049],
        [-0.04305307, -0.04987746, -0.0366137 , ...,  0.01739735,
          0.00826229,  0.04193049],
        [-0.04305307, -0.04987746, -0.0366137 , ...,  0.01739735,
          0.00826229,  0.04193049]]], dtype=float32)>

These values might not mean much to us but they're what our computer sees each word as. When our model looks for patterns in different samples, these values will be updated as necessary.

If we review the shape of Embedded Version Tensor it is, `(1, 15, 128)`, it means that,
- 1: Is the quantity of sequences(sentences) we passed
- 15: is the `max_length` that we decided to normalize every sentence, if greater than 15 tokens/words then trim extra tokens or if less than 15 then pad it.
- 128: is the array size of each words, for example above sentence as `Now`, for this token the embedding tensor will look like,

In [17]:
sample_embed[0][0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([-0.01314962,  0.00719387,  0.02004207, -0.03999835,  0.04695079,
        0.01789708,  0.03059348,  0.02138252, -0.03624801,  0.00396798,
       -0.03985019, -0.04329523,  0.01500774, -0.03659517,  0.03917797,
       -0.04362958,  0.0154495 ,  0.0078933 , -0.01369306,  0.03302287,
       -0.00288031, -0.04553231,  0.04097218, -0.01234646, -0.04826026,
        0.02076019,  0.00254625, -0.00031816, -0.00150321, -0.00461284,
       -0.04060421, -0.03143873,  0.0298737 ,  0.03604234, -0.0433757 ,
       -0.01436574, -0.03462224,  0.0473052 , -0.04071296,  0.03264949,
       -0.03514206,  0.00570067, -0.03294205,  0.00819801,  0.02374965,
       -0.04027864,  0.04482595,  0.0060604 ,  0.03028399,  0.01870103,
       -0.03406491,  0.03605679,  0.02144077,  0.03187365, -0.04643034,
       -0.00920881,  0.04652185,  0.0316061 ,  0.04617418, -0.04849065,
       -0.02418332,  0.01280299,  0.00942918,  0.00199284, -0.03562398,
        0.020261

## Modelling a Text Dataset

For experiments with various machine learning model for text classifier we will be considering below experiments:
- **Model 0** : Naive Bayes (baseline)
- **Model 1** : Feed-forward neural network (dense model) 
- **Model 2** : LSTM Model (RNN)
- **Model 3** : GRU (RNN)
- **Model 4** : Bidirectional-LSTM (RNN)
- **Model 5** : 1D CNN
- **Model 6** : TF Hub Pre-trained Feature Extractor
- **Model 7** : Same as model 6 with 10% of training samples

### Model 0 : Getting a baseline

We'll use `scikit-learn` library for building this model, and create a Scikit-Learn Pipeline using the TF-IDF (term frequency-inverse document frequency) formula to convert words into numbers and then model them using Multinomial Naive Bayes Algorithm.

> 📖 **Reading**: About TF-IDF on [Scikit-learn documentation](https://scikit-learn.org/stable/modules/feature_extraction.html#tfidf-term-weighting)

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("clf", MultinomialNB())
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

Multinomial Naive Bayes model is kind of shallow model which trains faster.

In [19]:
# evaluate the model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 80.18%


In [20]:
# let's make some predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:10]

array([0, 0, 0, 0, 1, 1, 0, 1, 0, 0])

In [21]:
# it is similar to our labels
val_labels[:10]

array([0, 0, 0, 0, 1, 1, 0, 1, 0, 1])

## Evaluation function for our Model Experiments

There are various metrics to evaluate the classification models like `precision`, `f1-score`, `recall`. So let's look at them through a function altogether.

In [22]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
def calculate_results(model, y_true, y_pred):
  """
  Calculates Model accuracy, precision, recall and f1-score for a binary classification model
  """
  # calculate the model accuracy
  model_accuracy = accuracy_score(y_true, y_pred)
  # calculate precision, recall, f1-score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy * 100,
                   "precision":model_precision,
                   "recall": model_recall,
                   "f1-score": model_f1}
                  
  return model_results


In [24]:
# get the results
baseline_results = calculate_results(model=model_0,
                                     y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 80.18372703412074,
 'precision': 0.8125567744156732,
 'recall': 0.8018372703412073,
 'f1-score': 0.7968681002825004}